In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import spacy
import en_core_web_sm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt  
#from pandas_profiling import ProfileReport   
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, auc, roc_curve, precision_recall_curve
#pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df=pd.read_csv('/Users/anagha/Desktop/Sentiment Analysis/Womens Clothing E-Commerce Reviews.csv')

In [3]:
df.drop(['Age','Clothing ID', 'Division Name', 'Department Name', 'Class Name'], axis = 1, inplace=True)
df.reset_index(inplace=True)
df.drop(['Unnamed: 0'],axis = 1, inplace=True)
df.set_index('index')
index_names = df[df['Review Text'].isnull() & df['Title'].isnull()].index 
df.drop(index_names, inplace = True) 
df[df['Review Text'].isnull() & df['Title'].isnull()].sum(axis=0)
df['Review Text'].isnull().sum(axis = 0)
idx=df[df['Review Text'].isnull()].index
df['Title'].isnull().sum(axis = 0)
df['Review Text'][idx]
df['Title'][idx]
df['Review Text'][idx]=df['Title'][idx]
df['Review Text'][idx]
df.drop(['Title'],axis = 1, inplace=True)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [4]:
import unicodedata
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    

    return str(text)

df['Review Text Precrocessed']=df['Review Text'].apply(lambda x: strip_accents(x))

def char_count(x):
    s=x.split()
    s=str(s)
    x=' '.join(s)
    return len(x)

df['Char_Count']=df['Review Text Precrocessed'].apply(lambda x: char_count(x.encode('utf-8')))


def word_count(x):
    return len(str(x).split())

df['Word_Count']=df['Review Text Precrocessed'].apply(lambda x: word_count(x.encode('utf-8')))

def remove_sapce(x):
    x=' '.join(x.split())
    return x
df['Review Text Precrocessed']=df['Review Text Precrocessed'].apply(lambda x: remove_sapce(x))

df['Review Text Precrocessed']=df['Review Text Precrocessed'].str.lower()

contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"i'm": "i am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

for key, value in contractions.items():
  key= key.lower()
  value=value.lower()
  # print(key)
  # print(value)


def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value=contractions[key]
            x=x.replace(key,value)
        return x
    return x
df['Review Text Precrocessed']=df['Review Text Precrocessed'].apply(lambda x: cont_to_exp(x))


In [5]:
def label(x):
    if(x>3):
        return 1
    elif(x==3):
        return 2
    else:
        return 0

df['Label_3C']=df['Rating'].apply(lambda x:label(x))
Positive_review=[]
Negative_review=[]
Neutral_review=[]
Positive_review=df[df['Label_3C']==1]
Negative_review=df[df['Label_3C']==0]
Neutral_review=df[df['Label_3C']==2]


In [21]:
## Calculating TFIDF using sklearn

### Tfidf on preprocessed data

stopwords = ['a', 'about', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', \
             'even', 'ever', 'for', 'from', 'get', 'had', 'has', 'have', 'he', 'her', 'hers', 'his', \
             'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me', 'my', 'of', 'on', 'or', \
             'see', 'seen', 'she', 'so', 'than', 'that', 'the', 'their', 'there', 'they', 'this', \
             'to', 'was', 'we', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'you']

tfidf_3_class = TfidfVectorizer(stop_words=stopwords)
X_3_class = tfidf_3_class.fit_transform(df['Review Text Precrocessed'])
print(X_3_class.shape)


print(df['Label_3C'].value_counts())
print(df['Review Text Precrocessed'].count())

# ## Splitting data

# ### Specify the Label and Features

y_3c=df['Label_3C']
X_3c=X_3_class

print(y_3c.shape)
print(X_3c.shape)

from sklearn import preprocessing
X_scaled = preprocessing.normalize(X_3c, norm='l2')

X_train_3c, X_test_3c, y_train_3c, y_test_3c = train_test_split(X_3c ,y_3c ,test_size=.20, random_state=101)

print(df['Label_3C'].value_counts())

(22642, 14084)
1    17449
2     2823
0     2370
Name: Label_3C, dtype: int64
22642
(22642,)
(22642, 14084)
1    17449
2     2823
0     2370
Name: Label_3C, dtype: int64


In [1]:
y_test_3c

NameError: name 'y_test_3c' is not defined

In [52]:
print("Build random Forest Classifier ")

def RandomForest(X_train_3c, y_train_3c, X_test_3c, y_test_3c):
    clf_3c = RandomForestClassifier()
    clf_3c.fit(X_train_3c, y_train_3c)
    y_pred_3c = clf_3c.predict(X_test_3c)
    cm_RFC=confusion_matrix(y_test_3c, y_pred_3c)
    print(confusion_matrix(y_test_3c, y_pred_3c))
    print(classification_report(y_test_3c, y_pred_3c)) 
    print(accuracy_score(y_test_3c, y_pred_3c))
    return cm_RFC

RandomForest(X_train_3c, y_train_3c, X_test_3c, y_test_3c)

Build random Forest Classifier 
[[  99  351   20]
 [  27 3455   16]
 [  54  480   27]]
             precision    recall  f1-score   support

          0       0.55      0.21      0.30       470
          1       0.81      0.99      0.89      3498
          2       0.43      0.05      0.09       561

avg / total       0.73      0.79      0.73      4529

0.7906822698167366


array([[  99,  351,   20],
       [  27, 3455,   16],
       [  54,  480,   27]])

In [41]:
print("Build random Forest Classifier ")

clf_3c = RandomForestClassifier(min_samples_leaf = 2, n_estimators =15, min_samples_split =8, criterion= 'gini', max_features=2000, max_depth= 39)
clf_3c.fit(X_train_3c, y_train_3c)
y_pred_3c = clf_3c.predict(X_test_3c)
cm_RFC=confusion_matrix(y_test_3c, y_pred_3c)
print(confusion_matrix(y_test_3c, y_pred_3c))
print(classification_report(y_test_3c, y_pred_3c)) 
print(accuracy_score(y_test_3c, y_pred_3c))

Build random Forest Classifier 
[[ 124  293   53]
 [  39 3415   44]
 [  65  424   72]]
             precision    recall  f1-score   support

          0       0.54      0.26      0.36       470
          1       0.83      0.98      0.90      3498
          2       0.43      0.13      0.20       561

avg / total       0.75      0.80      0.75      4529

0.7973062486200044


In [30]:
############### Hyperparameter tuning for Random Forest with Randomised Search

from scipy.stats import randint 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import uniform, truncnorm, randint
  
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

model_params = {
              "n_estimators" : [10, 12, 15],
              "max_depth": [10, 18, 20, 25], 
              "max_features": randint(1, 3000), 
              "min_samples_leaf": randint(1, 12), 
              "criterion": ["gini", "entropy"],
              'min_samples_split': [2, 4, 6, 8, 10]} 


tree = RandomForestClassifier()
tree_cv =  RandomizedSearchCV(tree, model_params, n_iter=100, cv=cv, random_state=1)
tree_fit = tree_cv.fit(X_train_3c, y_train_3c) 
tree_predict = tree_fit.predict(X_test_3c)

print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_)) 
print("Best score is {}".format(tree_cv.best_score_)) 

print(confusion_matrix(y_test_3c, tree_predict))
print(classification_report(y_test_3c, tree_predict)) 
print(accuracy_score(y_test_3c, tree_predict))

Tuned Decision Tree Parameters: {'min_samples_leaf': 1, 'n_estimators': 12, 'min_samples_split': 8, 'criterion': 'gini', 'max_features': 1494, 'max_depth': 25}
Best score is 0.788733690351
[[ 114  312   44]
 [  41 3429   28]
 [  62  458   41]]
             precision    recall  f1-score   support

          0       0.53      0.24      0.33       470
          1       0.82      0.98      0.89      3498
          2       0.36      0.07      0.12       561

avg / total       0.73      0.79      0.74      4529

0.7913446676970634


In [50]:
############### Hyperparameter tuning for Random Forest Grid CV

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

rfc=RandomForestClassifier(random_state=42)

# param_grid = { 
#     'n_estimators': [100,200,300,400,500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }

param_grid = {"n_estimators" : [10, 12, 15],
             "max_depth": [10, 18, 20, 25, 30, 35, 40, 45, 50], 
             'max_features': ['auto', 'sqrt', 'log2'], 
              "min_samples_leaf": [1, 2, 4, 6, 8, 12],
             "criterion": ["gini", "entropy"],
              'min_samples_split': [2, 4, 6, 8, 10]
        } 


#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc = GridSearchCV(rfc, param_grid=param_grid, cv= 5)
grid_tree =CV_rfc.fit(X_train_3c, y_train_3c)
grid_tree =CV_rfc.fit(X_train_3c, y_train_3c)
grid_predict = grid_tree.predict(X_test_3c)

print(CV_rfc.best_params_)
means = CV_rfc.cv_results_['mean_test_score']

print(confusion_matrix(y_test_3c, grid_predict))
print(classification_report(y_test_3c, grid_predict)) 
print(accuracy_score(y_test_3c, grid_predict))

KeyboardInterrupt: 

In [ ]:
'max_features': 2268, 'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 2

In [ ]:
Tuned Decision Tree Parameters: {'max_features': 2963, 'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 5}
Best score is 0.782145420416
[[  73  372   25]
 [  19 3473    6]
 [  39  490   32]]
             precision    recall  f1-score   support

          0       0.56      0.16      0.24       470
          1       0.80      0.99      0.89      3498
          2       0.51      0.06      0.10       561

avg / total       0.74      0.79      0.72      4529

0.7900198719364098

In [ ]:
from scipy.stats import randint 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import uniform, truncnorm, randint
  
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

model_params = {
                "max_depth": [10, 18], 
              "max_features": randint(1, 3000), 
              "min_samples_leaf": randint(1, 12), 
              "criterion": ["gini", "entropy"],
              'min_samples_split': [2, 4, 6]} 


tree = RandomForestClassifier()
tree_cv =  RandomizedSearchCV(tree, model_params, n_iter=100, cv=cv, random_state=1)
tree_fit = tree_cv.fit(X_train_3c, y_train_3c) 
tree_predict = tree_fit.predict(X_test_3c)

print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_)) 
print("Best score is {}".format(tree_cv.best_score_)) 

print(confusion_matrix(y_test_3c, tree_predict))
print(classification_report(y_test_3c, tree_predict)) 
print(accuracy_score(y_test_3c, tree_predict))



Tuned Decision Tree Parameters: {'max_features': 2950, 'min_samples_split': 6, 'criterion': 'gini', 'max_depth': 18, 'min_samples_leaf': 2}
Best score is 0.786930197464
[[ 114  320   36]
 [  45 3426   27]
 [  59  463   39]]
             precision    recall  f1-score   support

          0       0.52      0.24      0.33       470
          1       0.81      0.98      0.89      3498
          2       0.38      0.07      0.12       561

avg / total       0.73      0.79      0.74      4529

0.7902406712298521

In [ ]:
from scipy.stats import randint 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import uniform, truncnorm, randint
  
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

model_params = {
              "n_estimators" : [10, 12, 15],
             "max_depth": [10, 18, 20, 25], 
              "max_features": randint(1, 3000), 
              "min_samples_leaf": randint(1, 12), 
              "criterion": ["gini", "entropy"],
              'min_samples_split': [2, 4, 6, 8, 10]} 


tree = RandomForestClassifier()
tree_cv =  RandomizedSearchCV(tree, model_params, n_iter=100, cv=cv, random_state=1)
tree_fit = tree_cv.fit(X_train_3c, y_train_3c) 
tree_predict = tree_fit.predict(X_test_3c)

print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_)) 
print("Best score is {}".format(tree_cv.best_score_)) 

print(confusion_matrix(y_test_3c, tree_predict))
print(classification_report(y_test_3c, tree_predict)) 
print(accuracy_score(y_test_3c, tree_predict))


Tuned Decision Tree Parameters: {'min_samples_leaf': 1, 'n_estimators': 12, 'min_samples_split': 8, 'criterion': 'gini', 'max_features': 1494, 'max_depth': 25}
Best score is 0.788733690351
[[ 114  312   44]
 [  41 3429   28]
 [  62  458   41]]
             precision    recall  f1-score   support

          0       0.53      0.24      0.33       470
          1       0.82      0.98      0.89      3498
          2       0.36      0.07      0.12       561

avg / total       0.73      0.79      0.74      4529

0.7913446676970634